### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [17]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [18]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

np.float64(0.5854345727938506)

----
### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#### Item Nº 1:
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [31]:
# Tomo 5 documentos aleatorios para luego medir la similutud de estos.

import random

num_documents = 5
random_idx = random.sample(range(X_train.shape[0]), num_documents)
print(random_idx)

[1269, 8650, 5099, 6187, 9660]


In [44]:
for idx in random_idx:
    print(f"\n\033[33m==========================================\033[0m")
    print(f"\033[92mDocumento indice: {idx}\033[0m")
    print(newsgroups_train.data[idx])
    print(f"Etiqueta: {newsgroups_train.target_names[y_train[idx]]}\n")

    # Calculo la similaridad coseno entre el documento seleccionado y todos los demas documentos
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Calculo los 5 documentos mas similares (excluyendo el documento original)
    mostsim = np.argsort(cossim)[::-1][1:6]

    print(f"\033[92mDocumentos mas similares para el indice: {idx}\033[0m")
    j = 1
    for i in mostsim:
        print(f"\033[92mÍndice: {i}, Similaridad: {cossim[i]}, documento {j}/5\033[0m")
        print(newsgroups_train.data[i])
        print(f"Etiqueta: {newsgroups_train.target_names[y_train[i]]}\n")
        j += 1


Documento indice: 1269



I have the same card, TVGA-8900c. When I checked, the latest driver for
windows 3.1 is dated Aug. 92 in garbo.uwasa.fi in /win31/drivers/video.
If you find a better version (updated) please let me know, Thanks.

Etiqueta: comp.os.ms-windows.misc

Documentos mas similares para el indice: 1269
Índice: 3283, Similaridad: 0.5743235173780614, documento 1/5



Since we are in the subject, I have one more question. I have a Trident
8900C Video Card. I want to know what is the latest video driver for it.
So far, all I can find is that an old driver dated Aug. 92 in garbo.uwasa.fi
Anyone have any info ? Please e-mail me at axh113@psuvm.psu.edu
Etiqueta: comp.os.ms-windows.misc

Índice: 7669, Similaridad: 0.3139757811973024, documento 2/5
Hi, I have a trident TVGA-8900 video card and need the updated
drivers for Win3.1 where can I get them from an ftp site.


					Thanks
Etiqueta: comp.os.ms-windows.misc

Índice: 8449, Similaridad: 0.2698859094970669, documento 3/5
Hel

----
### Item Nº 2

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


In [45]:
from sklearn.model_selection import GridSearchCV

# Defino el vectorizador y los modelos
tfidfvect = TfidfVectorizer()
clf_mnb = MultinomialNB()
clf_cnb = ComplementNB()

In [53]:
# Defino un pipeline para unir el vectorizador y el modelo
from sklearn.pipeline import Pipeline

pipeline_mnb = Pipeline([
    ('tfidf', tfidfvect),
    ('clf', clf_mnb)
])

pipeline_cnb = Pipeline([
    ('tfidf', tfidfvect),
    ('clf', clf_cnb)
])

# Defino los parámetros para búsqueda en cuadrícula
param_grid = {
    'tfidf__max_df': [0.75, 0.85, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__alpha': [0.1, 0.5, 1.0]
}

# Armo el gridsearch para ambos modelos
grid_mnb = GridSearchCV(pipeline_mnb, param_grid, scoring='f1_macro', cv=5)
grid_cnb = GridSearchCV(pipeline_cnb, param_grid, scoring='f1_macro', cv=5)

grid_mnb.fit(newsgroups_train.data, y_train)
grid_cnb.fit(newsgroups_train.data, y_train)

# Evaluar conjunto de test
y_pred_mnb = grid_mnb.best_estimator_.predict(newsgroups_test.data)
y_pred_cnb = grid_cnb.best_estimator_.predict(newsgroups_test.data)

# F1-score macro
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')

print(f"Mejor F1-score Macro MultinomialNB: {f1_mnb}")
print(f"Mejor F1-score Macro ComplementNB: {f1_cnb}")

Mejor F1-score Macro MultinomialNB: 0.6577923576970611
Mejor F1-score Macro ComplementNB: 0.7058374784034724


Los resultados muestran que el modelo ComplementNB superó a MultinomialNB en la tarea de clasificación, obteniendo un mayor f1-score macro (0.7058 frente a 0.6578). Esto sugiere que ComplementNB es más efectivo en este conjunto de datos

----
### Ítem Nº 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [52]:
# Ajusto el vectorizador primero para generar el atributo vocabulary_
tfidfvect.fit(newsgroups_train.data)

# Transpongo la matriz
X_train_T = X_train.T
print(f"Shape de X_train_T: {X_train_T.shape}")

# Eligo 5 palabras manualmente (asegurándonos que sean interpretables)
palabras = ['car', 'was', 'game', 'space', 'windows']

for palabra in palabras:
    print(f"\033[41m\nPalabra: {palabra}\033[0m")

    # Obtenemos el índice de la palabra en el vocabulario
    idx = tfidfvect.vocabulary_.get(palabra, None)
    
    if idx is not None:
        # Calculamos la similaridad coseno entre la palabra seleccionada y todas las demás
        cossim = cosine_similarity(X_train_T[idx], X_train_T)[0]
        
        # Obtenemos las 5 palabras más similares
        mostsim = np.argsort(cossim)[::-1][1:6]
        
        print("Palabras más similares:")
        for i in mostsim:
            print(f"{idx2word[i]} (similaridad: {cossim[i]})")
    else:
        print(f"La palabra '{palabra}' no está en el vocabulario.")

Shape de X_train_T: (101631, 11314)

Palabra: car
Palabras más similares:
cars (similaridad: 0.179666464839714)
criterium (similaridad: 0.17704033317887863)
civic (similaridad: 0.17477335763813667)
owner (similaridad: 0.16890630770475856)
dealer (similaridad: 0.16811115268527982)

Palabra: was
Palabras más similares:
the (similaridad: 0.4832985185623878)
that (similaridad: 0.4219570038510486)
of (similaridad: 0.4109529286401817)
in (similaridad: 0.4079755227493348)
and (similaridad: 0.40232289724437853)

Palabra: game
Palabras más similares:
games (similaridad: 0.21162262820656594)
espn (similaridad: 0.1935104386908168)
hockey (similaridad: 0.18061742766642597)
team (similaridad: 0.17862365088802803)
scored (similaridad: 0.170710357682652)

Palabra: space
Palabras más similares:
nasa (similaridad: 0.33042209494811214)
seds (similaridad: 0.29664333833449597)
shuttle (similaridad: 0.29284482622873)
enfant (similaridad: 0.2802695315352051)
seti (similaridad: 0.24649360843797932)

Palabra:

Los resultados muestran que la similaridad del coseno entre palabras capturó correctamente relaciones semánticas relevantes en cada caso. Las palabras seleccionadas ("car", "was", "game", "space", "windows") tienen términos asociados coherentes con su significado, lo que indica que la vectorización refleja adecuadamente el contexto y la temática de los documentos analizados.